In [39]:
import json
import pickle
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [40]:
DATA_DIR = Path("data")
RECIPES_DATA = DATA_DIR / "recipe-ingredients-dataset.csv"

In [41]:
df = pd.read_csv(RECIPES_DATA)[:2500]

In [42]:
df["tags_str"] = df["tags"].apply(lambda x: " ".join(json.loads(x.replace('\'', '"').replace('{', '[').replace('}', ']'))))
df["search_terms_str"] = df["search_terms"].apply(lambda x: " ".join(json.loads(x.replace('\'', '"').replace('{', '[').replace('}', ']'))))
df["ingredients_str"] = df["ingredients"].apply(lambda x: " ".join(json.loads(x.replace('\'', '"'))))

In [43]:
df

,Unnamed: 0,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms,cuisine,tags_str,search_terms_str,ingredients_str
0,0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to ser...","['water', 'grits', 'salt', 'cheddar cheese', '...","[""4 cups water"",""1 cup uncooked old f...",1 (155 g),8,"['I a sauce pan, bring water to a boil; slowly...","['time-to-make', 'course', 'main-ingredient', ...","{'diabetic', 'low-calorie', 'vegetarian', 'low...",southern_us,time-to-make course main-ingredient preparatio...,diabetic low-calorie vegetarian low-carb side,water grits salt cheddar cheese garlic olive oil
1,1,232037,Simple Shrimp and Andouille Jambalaya,"Simple, easy and very tasty for when you are i...","['onion', 'red bell pepper', 'garlic cloves', ...","[""1 medium onion, chopped coarse "",""1 m...",1 (366 g),4,"['In a food processor, pulse the onion, red pe...","['60-minutes-or-less', 'time-to-make', 'course...","{'dinner', 'shrimp'}",cajun_creole,60-minutes-or-less time-to-make course main-in...,dinner shrimp,onion red bell pepper garlic cloves large shri...
2,2,41090,black-and-white bean salad,NaN,"['white beans', 'canned black beans', 'tomatoe...","[""1 cup canned white beans, rinsed and dra...",1 (807 g),1,"['In a large bowl, combine beans, tomato, onio...","['15-minutes-or-less', 'time-to-make', 'course...","{'vegetarian', 'salad', 'side', 'dinner', 'veg...",italian,15-minutes-or-less time-to-make course main-in...,vegetarian salad side dinner vegan,white beans canned black beans tomatoes onion ...
3,3,60656,Crock Pot Italian Zucchini,This is a good recipe for weight watchers. It ...,"['zucchini', 'yellow squash', 'diced tomatoes'...","[""2 zucchini, sliced "",""2 small yel...",1 (244 g),4,['Put all ingredients in the crock pot and coo...,"['weeknight', 'time-to-make', 'course', 'main-...","{'side', 'vegetarian', 'italian'}",italian,weeknight time-to-make course main-ingredient ...,side vegetarian italian,zucchini yellow squash diced tomatoes onion ga...
4,4,232047,Beef Stew With Dried Cherries,This is a fabulous stew that came from one of ...,"['beef stew meat', 'flour', 'salt', 'allspice'...","[""3 lbs beef stew meat"",""3 tablespoons ...",1 (358 g),8,"['Preheat oven to 350°F.', ""Cut beef into 1 in...","['time-to-make', 'course', 'main-ingredient', ...",{'dinner'},russian,time-to-make course main-ingredient preparatio...,dinner,beef stew meat flour salt allspice cinnamon bl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,2495,350629,Belgian White Beer Cookies With Orange Icing,I haven't tried this recipe yet but it sounds ...,"['beer', 'honey', 'unsalted butter', 'powdered...","["""",""2 (12 ounce) bottles belgian style whi...",1 (133 g),12,['Put the beer and honey to a medium saucepan ...,"['60-minutes-or-less', 'time-to-make', 'course...","{'cookie', 'dessert'}",cajun_creole,60-minutes-or-less time-to-make course prepara...,cookie dessert,beer honey unsalted butter powdered sugar egg ...
2496,2496,397738,Homestyle Lamb Shanks,"I'm a HUGE Triple D (Diner's, Drive-In's & Div...","['lamb shanks', 'extra virgin olive oil', 'bee...","[""4 lamb shanks"",""4 -6 tablespoons ...",1 (742 g),4,"['The night before, season the lamb shanks wit...","['course', 'main-ingredient', 'main-dish', 'la...","{'lamb', 'dinner'}",greek,course main-ingredient main-dish lamb-sheep meat,lamb dinner,lamb shanks extra virgin olive oil beef broth ...
2497,2497,416351,Peppermint Patties,These Peppermint Patties are both Vegan and Gl...,"['coconut oil', 'agave nectar', 'peppermint oi...","[""1/2 cup coconut oil"",""1/4 cup agave ...",1 (29 g),4,"['In a small bowl, combine coconut oil, agave ...","['15-minutes-or-less', 'time-to-make', 'course...","{'dessert', 'gluten-free'}",brazilian,15-minutes-or-less time-to-make course prepara...,dessert gluten-free,coconut oil agave nectar peppermint oil dark c...
2498,2498,416432,Veggie Crumble Tacos,I sometimes get a cravi

In [44]:
df["all_tags"] = df["description"] + " " + df["ingredients_str"] + " " + df["tags_str"] + " " + df["search_terms_str"] + " " + df["cuisine"]

In [45]:
df

,Unnamed: 0,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms,cuisine,tags_str,search_terms_str,ingredients_str,all_tags
0,0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to ser...","['water', 'grits', 'salt', 'cheddar cheese', '...","[""4 cups water"",""1 cup uncooked old f...",1 (155 g),8,"['I a sauce pan, bring water to a boil; slowly...","['time-to-make', 'course', 'main-ingredient', ...","{'diabetic', 'low-calorie', 'vegetarian', 'low...",southern_us,time-to-make course main-ingredient preparatio...,diabetic low-calorie vegetarian low-carb side,water grits salt cheddar cheese garlic olive oil,"We love grits, this is another good way to ser..."
1,1,232037,Simple Shrimp and Andouille Jambalaya,"Simple, easy and very tasty for when you are i...","['onion', 'red bell pepper', 'garlic cloves', ...","[""1 medium onion, chopped coarse "",""1 m...",1 (366 g),4,"['In a food processor, pulse the onion, red pe...","['60-minutes-or-less', 'time-to-make', 'course...","{'dinner', 'shrimp'}",cajun_creole,60-minutes-or-less time-to-make course main-in...,dinner shrimp,onion red bell pepper garlic cloves large shri...,"Simple, easy and very tasty for when you are i..."
2,2,41090,black-and-white bean salad,NaN,"['white beans', 'canned black beans', 'tomatoe...","[""1 cup canned white beans, rinsed and dra...",1 (807 g),1,"['In a large bowl, combine beans, tomato, onio...","['15-minutes-or-less', 'time-to-make', 'course...","{'vegetarian', 'salad', 'side', 'dinner', 'veg...",italian,15-minutes-or-less time-to-make course main-in...,vegetarian salad side dinner vegan,white beans canned black beans tomatoes onion ...,NaN
3,3,60656,Crock Pot Italian Zucchini,This is a good recipe for weight watchers. It ...,"['zucchini', 'yellow squash', 'diced tomatoes'...","[""2 zucchini, sliced "",""2 small yel...",1 (244 g),4,['Put all ingredients in the crock pot and coo...,"['weeknight', 'time-to-make', 'course', 'main-...","{'side', 'vegetarian', 'italian'}",italian,weeknight time-to-make course main-ingredient ...,side vegetarian italian,zucchini yellow squash diced tomatoes onion ga...,This is a good recipe for weight watchers. It ...
4,4,232047,Beef Stew With Dried Cherries,This is a fabulous stew that came from one of ...,"['beef stew meat', 'flour', 'salt', 'allspice'...","[""3 lbs beef stew meat"",""3 tablespoons ...",1 (358 g),8,"['Preheat oven to 350°F.', ""Cut beef into 1 in...","['time-to-make', 'course', 'main-ingredient', ...",{'dinner'},russian,time-to-make course main-ingredient preparatio...,dinner,beef stew meat flour salt allspice cinnamon bl...,This is a fabulous stew that came from one of ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,2495,350629,Belgian White Beer Cookies With Orange Icing,I haven't tried this recipe yet but it sounds ...,"['beer', 'honey', 'unsalted butter', 'powdered...","["""",""2 (12 ounce) bottles belgian style whi...",1 (133 g),12,['Put the beer and honey to a medium saucepan ...,"['60-minutes-or-less', 'time-to-make', 'course...","{'cookie', 'dessert'}",cajun_creole,60-minutes-or-less time-to-make course prepara...,cookie dessert,beer honey unsalted butter powdered sugar egg ...,I haven't tried this recipe yet but it sounds ...
2496,2496,397738,Homestyle Lamb Shanks,"I'm a HUGE Triple D (Diner's, Drive-In's & Div...","['lamb shanks', 'extra virgin olive oil', 'bee...","[""4 lamb shanks"",""4 -6 tablespoons ...",1 (742 g),4,"['The night before, season the lamb shanks wit...","['course', 'main-ingredient', 'main-dish', 'la...","{'lamb', 'dinner'}",greek,course main-ingredient main-dish lamb-sheep meat,lamb dinner,lamb shanks extra virgin olive oil beef broth ...,"I'm a HUGE Triple D (Diner's, Drive-In's & Div..."
2497,2497,416351,Peppermint Patties,These Peppermint Patties are both Vegan and Gl...,"['coconut oil', 'agave nectar', 'peppermint oi...","[""1/2 cup coconut oil"",""1/4 cup agave ...",1 (29 g),4,"

In [46]:
df.drop(df[df['all_tags'].isnull()].index, inplace=True)
df = df.reset_index()

In [47]:
tfidf = TfidfVectorizer(max_features=5000, stop_words="english")

# Transform the data
vectorized_data = tfidf.fit_transform(df['all_tags'].values)
vectorized_dataframe = pd.DataFrame(vectorized_data.toarray(), index=df['all_tags'].index.tolist())
vectorized_dataframe.shape

(2458, 5000)

In [48]:
svd = TruncatedSVD(n_components=3000)

# Fit transform the data
reduced_data = svd.fit_transform(vectorized_dataframe)
reduced_data

array([[ 3.94126563e-01,  2.63226903e-02,  4.52580905e-02, ...,
        -2.20871901e-03, -3.51726827e-03,  9.73048245e-04],
       [ 2.17704449e-01, -2.10208091e-01,  1.45675617e-01, ...,
        -9.42674436e-04,  2.91896114e-04, -3.05710186e-04],
       [ 2.23439375e-01, -1.78094335e-01,  9.94855391e-02, ...,
        -1.55941935e-03,  2.09643988e-03,  5.01416076e-03],
       ...,
       [ 1.01100073e-01, -7.60550528e-02, -9.96711079e-02, ...,
        -5.96682302e-04,  4.88817897e-04, -2.67160832e-04],
       [ 4.47958840e-01,  1.32182628e-01,  7.98279245e-02, ...,
        -9.93246748e-05,  2.59247968e-04, -3.96883565e-04],
       [ 2.16196599e-01, -1.07428809e-01,  5.91094758e-02, ...,
        -2.43749991e-04, -5.99733859e-04, -7.48787817e-04]])

In [49]:
def recommendation(prompt, k=5):
    target_vector = tfidf.transform([prompt])
    reduced_target_vector = svd.transform(target_vector)

    similarities = cosine_similarity(reduced_target_vector, reduced_data).flatten()
    topk = np.argpartition(similarities, -k)[-k:]
    
    return [(df.iloc[i]["name"]) for i in topk]

In [50]:

recommendation("a type of sandwich that uses one piece continuous piece of bread that cradles a protein a fiber and a sort of dressing. something I don't have to cook")

['Low Fat Peanut Butter Chocolate Crunch Bars',
 'Japanese Cafeteria Spaghetti Sauce',
 'The Godfather of Grilled Cheese Sandwiches',
 'Double Chocolate Chunk Cookies',
 'Chocolate Nut Balls']

In [51]:
dish_prompts = [
    "I'm looking for a dairy-free breakfast dish with a hint of spice; something like a Mexican-style omelet but without cheese.",
    "Can you suggest a gluten-free Italian lunch option, preferably with a lot of fresh vegetables and no meat?",
    "I need a quick Japanese-inspired snack, but I'm allergic to shellfish—something light and vegetarian would be ideal.",
    "I'm craving a hearty vegan dinner that's Mediterranean-inspired, featuring chickpeas or lentils and lots of fresh herbs.",
    "Could you recommend a kid-friendly dessert that's nut-free but still has a touch of chocolate? Maybe something from French cuisine?",
    "I want a light Thai lunch dish, but it has to be free of soy and seafood, with a focus on fresh and crisp ingredients.",
    "I'm in the mood for a comforting American breakfast but need it to be egg-free—something like pancakes or waffles without eggs would be great.",
    "Give me an Indian-inspired dinner option that's mild in spice and free of dairy. I'd love something with a good mix of veggies.",
    "I need a quick snack that's Mediterranean-style and free of gluten and dairy, ideally something I can eat on the go.",
    "I'm looking for a festive dessert for dinner, inspired by Mexican cuisine but without any cinnamon or nuts."
]

In [52]:
for prompt in dish_prompts:
    print(prompt)
    results = recommendation(prompt)
    for i, dish in enumerate(results, start=1):
        print(f"\t{i}:", dish)
    print()

I'm looking for a dairy-free breakfast dish with a hint of spice; something like a Mexican-style omelet but without cheese.
	1: Low Calorie Pancakes
	2: Gluten- and Dairy-Free Chicken Enchiladas
	3: Cheesy Eggs &amp; Rice
	4: Southwestern Omelet Wrap - Ziploc Steam Bags
	5: Quinoa Rice Breakfast (Gluten Free, Dairy Free)

Can you suggest a gluten-free Italian lunch option, preferably with a lot of fresh vegetables and no meat?
	1: Ginger, Fig and Shallot Conserve
	2: Quinoa Rice Breakfast (Gluten Free, Dairy Free)
	3: Costa Rican Chilaquiles (Tortilla &amp; Meat)
	4: Fun Mini Muffins -Almond Meal- Flourless
	5: Italian Herb Salad Dressing

I need a quick Japanese-inspired snack, but I'm allergic to shellfish—something light and vegetarian would be ideal.
	1: Miso-Garlic Broiled Eggplant
	2: Garnished Rice With Green Tea (Ocha-Zuke)
	3: Light and Crisp Veggie Sandwich
	4: Carrot and Sesame Furikake
	5: Japanese Spring Vegetable Ragout

I'm craving a hearty vegan dinner that's Mediterran

In [53]:
# save
with open('models/recipe_svd.pkl','wb') as f:
    pickle.dump(svd,f)

# load
with open('models/recipe_svd.pkl', 'rb') as f:
    svd = pickle.load(f)

In [57]:
# save
with open('models/recipe_vectors.pkl','wb') as f:
    pickle.dump(reduced_data,f)

# load
with open('models/recipe_vectors.pkl', 'rb') as f:
    reduced_data = pickle.load(f)

In [54]:
# save
with open('vectorizers/recipe_vectorizer.pkl','wb') as f:
    pickle.dump(tfidf,f)

# load
with open('vectorizers/recipe_vectorizer.pkl', 'rb') as f:
    tfidf = pickle.load(f)

In [56]:
df.drop(columns=["index", "id", "Unnamed: 0"]).to_csv(DATA_DIR / "vectorized_recipes.csv")